In [ ]:
!pip install lightning
!pip install pytorch-lightning
!pip install matplotlib
!pip install torchvision

In [26]:
import ssl
import torch
import torchmetrics
from torch import nn
import lightning as L
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import Imagenette
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

# Basic CNN

In [31]:
class BasicCNN(L.LightningModule):
    def __init__(self, num_classes=10, size=8, in_channels=1):
        super().__init__()

        # Convolutional Layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully Connected Layers
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * size * size, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.train_losses = []
        self.val_losses = []


    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_losses.append(loss.item())
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)
        self.val_losses.append(loss.item())
        self.log("val_loss", loss)
        self.log("val_accuracy", self.accuracy)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_loss", loss)
        self.log("test_accuracy", self.accuracy)

    def train_epoch(self):
        avg_train_loss = sum(self.train_losses)/len(self.train_losses) if self.train_losses else None
        print(f'Epoch {self.current_epoch}: Average Training Loss:{avg_train_loss}')

    def validation_epoch(self):
        avg_val_loss = sum(self.val_losses)/len(self.val_losses) if self.val_losses else None
        print(f'Epoch {self.current_epoch}: Average Validation Loss:{avg_val_loss}')

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# ResNet CNN

In [32]:
class ResNet18(L.LightningModule):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()

        # Initial convolutional layer
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Convolutional layers without residual connections
        self.layer1 = self._make_layer(64, 64, stride=1)
        self.layer2 = self._make_layer(64, 128, stride=2)
        self.layer3 = self._make_layer(128, 256, stride=2)
        self.layer4 = self._make_layer(256, 512, stride=2)

        # Fully connected layer for classification
        self.fc = nn.Linear(512, num_classes)

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.train_losses = []
        self.val_losses = []

    def _make_layer(self, in_channels, out_channels, stride):
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_losses.append(loss.item())
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.val_losses.append(loss.item())
        self.log("val_accuracy", self.accuracy, prog_bar=True)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("test_accuracy", self.accuracy, prog_bar=True)
        self.log("test_loss", loss)

    def train_epoch(self):
        avg_train_loss = sum(self.train_losses)/len(self.train_losses) if self.train_losses else None
        print(f'Epoch {self.current_epoch}: Average Training Loss:{avg_train_loss}')

    def validation_epoch(self):
        avg_val_loss = sum(self.val_losses)/len(self.val_losses) if self.val_losses else None
        print(f'Epoch {self.current_epoch}: Average Validation Loss:{avg_val_loss}')

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# Regularization CNN

In [33]:
class RegularizationCNN(L.LightningModule):
    def __init__(self, num_classes=10, size=8):
        super().__init__()

        # Convolutional Layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully Connected Layers
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * size * size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(256, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.train_losses = []
        self.val_losses = []

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_losses.append(loss.item())
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)
        self.val_losses.append(loss.item())
        self.log("val_loss", loss)
        self.log("val_accuracy", self.accuracy)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_loss", loss)
        self.log("test_accuracy", self.accuracy)

    def train_epoch(self):
        avg_train_loss = sum(self.train_losses)/len(self.train_losses) if self.train_losses else None
        print(f'Epoch {self.current_epoch}: Average Training Loss:{avg_train_loss}')

    def validation_epoch(self):
        avg_val_loss = sum(self.val_losses)/len(self.val_losses) if self.val_losses else None
        print(f'Epoch {self.current_epoch}: Average Validation Loss:{avg_val_loss}')

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# Prepare the dataset

In [34]:
# Prepare the dataset
train_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])
ssl._create_default_https_context = ssl._create_unverified_context
train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=True, transform=train_transforms)

train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_set_size, val_set_size], generator=seed)
val_dataset.dataset.transform = test_transforms

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True, persistent_workers=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False, persistent_workers=True)

test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=True, transform=test_transforms)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, num_workers=8, shuffle=False)

# Basic CNN Model

In [35]:
basic_cnn_model = BasicCNN()

basic_cnn_early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

basic_cnn_checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    dirpath="./checkpoints",
    filename="model_weights",
    save_top_k=1,
    verbose=True
)

# Train the Basic CNN

In [37]:
basic_cnn_trainer = L.Trainer(callbacks=[basic_cnn_early_stop_callback, basic_cnn_checkpoint_callback], max_epochs=10)
trainer.fit(model=basic_cnn_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name        | Type               | Params | Mode 
-----------------------------------------------------------
0 | conv_layers | Sequential         | 92.7 K | train
1 | fc_layers   | Sequential         | 2.1 M  | train
2 | accuracy    | MulticlassAccuracy | 0      | train
-----------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.771     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 67/67 [00:01<00:00, 37.79it/s, v_num=9]          
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 9: 100%|██████████| 67/67 [00:01<00:00, 34.45it/s, v_num=9]      

Epoch 9, global step 670: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 67/67 [00:01<00:00, 34.37it/s, v_num=9]


# Test the Basic CNN

In [ ]:
basic_cnn_trainer.test(model=basic_cnn_model, dataloaders=test_loader)

# Plot training and validation losses

In [ ]:
# Plot training and validation losses
def plot_losses(model):
    plt.figure(figsize=(10, 5))
    plt.plot(model.train_losses, label="Training Loss")
    plt.plot(model.val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss over Epochs")
    plt.legend()
    plt.show()

plot_losses(basic_cnn_model)

# ResNet 18 Model

In [36]:
resnet_18_model = ResNet18()

early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

resnet_18_checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)